# 10.3 Distance Metrics

In the last section, we used Cosine Similarity to measure how similar two documents are. However, there are many other measures of similarity that exists. It will be up to us to decide which metric is best for our application. In this section, we will explore these other methods for measuring text similarity. 

## Hamming Distance

The most simple and intuitive method for measuring string similarity is the hamming distance. Given two strings of equal lengths, it measures the number of symbols that are different at each corresponding index. Take the example of the strings `"BRYAN"` and `"BRIAN"`. Here, we can see that the Hamming difference between the two strings is 1 because they are different at one location (third letter). 

In [0]:
def hamming_distance(str1, str2): 
    if len(str1) != len(str2): 
        return None 

    differences = 0
    for x in range(len(str1)): 
        if str1[x] != str2[x]: 
            differences += 1 
        
    return differences

hamming_distance("Bryan", "Brian") 

1

In theory, we can modify the Hamming Distance to accept two strings of different lengths, and whatever is left over is counted as a difference. The following is a simple implementation for this modified Hamming Distance

In [0]:
def modified_hamming_distance(str1, str2): 
    differences = 0

    if len(str1) != len(str2):
        differences += abs(len(str1) - len(str2))

    for x in range(len(str1)): 
        if str1[x] != str2[x]: 
            differences += 1 
        
    return differences

modified_hamming_distance("Bryan", "Brian!!!") 

4

## Edit Distance

A more applicable string metric is the edit distance. Generally, the edit distance is the smallest number of operations needed to turn one string into another. These operations may include insertions, deletions, substitutions, transposition, etc. Edit distances have different names based on their set of operations. For this chapter, we will focus on the Damerau-Levenshtein distance.

### Damerau-Levenshtein Distance

The Damerau-Levenshtein distance has four operations: deletion, insertion, substitution, and transposition. Here's what these operations mean:

* Deletion: **s**cat --> cat <br>
"scat" and "cat" have an edit distance of 1 because we can delete the s in "scat" to turn it into "cat". 

* Insertion: at --> **m**at <br>
"at" and "mat" have an edit distance of 1 because we can insert an m into "at" to turn it into "mat". 

* Substitution: **m**at --> **c**at <br>
"mat" and "cat" have an edit distance of 1 because we can substitute the m in "mat" for a c to turn "mat" into a "cat".

* Transposition: e**at** --> e**ta** <br>
"eat" and "eta" have an edit distance of 1 because we can transpose the successive characters of 'a' and 't' to turn "eat" into "eta". 

By measuring how many steps it takes to turn one string into another, it tells us how similar the two strings are. While the concept may be simple, it is infinitely applicable. One of the more interesting applications of the Damerau-Levenshtein distance is lingustic comparison. By computing the DL distance on two strings of different language but of same meaning, we can get a general idea of how similar the two languages are. 

Damerau-Levenshtein distance is conceptually simple but it is rather tedious to implement efficiently as it requires Dynamic Programming. NLTK provides us with an implementation of Damerau-Levenshtein distance with the `edit_distance()` function. Note that `edit_distance()` only provides the Levenshtein distance (without transposition). To make `edit_distance()` compute Damerau-Levenshtein distance we specify the parameter `transpositions = True`.


Below is an example of DL distance in practice using translation of an excerpt from Winston Churchill's famous "We shall fight on the beaches" speech into five different languages.


In [0]:
from nltk.metrics import edit_distance
import pandas as pd

df_speech = pd.read_csv("https://raw.githubusercontent.com/bfkwong/data/master/we_shall_never_surrender.csv", index_col="language")

def getDLDistance(language): 
    return edit_distance(df_speech.loc["english"]["text"], language, transpositions=True)

df_speech["Distance to English"] = df_speech["text"].apply(getDLDistance)
df_speech

,text,Distance to English
language,,
english,We shall go on to the end. We shall fight in F...,0
spanish,"Llegaremos hasta el final, lucharemos en Franc...",276
german,Wir werden bis zum Ende weitermachen. Wir werd...,334
portuguese,Iremos até ao fim. Lutaremos na França. Lutare...,281
french,"Nous irons jusqu'au bout, nous nous battrons e...",326
italian,ndremo avanti fino alla fine. Combatteremo in ...,292


When it comes to the vocabularies in this excerpt from Churchill's speech. English is most similar to Spanish, followed by Portuguese, Italian, German, and finally French. It is important that we don't extrapolate this conclusion to the entire language, as these results only shows that with the words and punctuations used in this speech.

## Jaccard Distance

This method of comparing string similarity is more based on statistics than pure comparison like Hamming Distance and Edit Distance. The formulation of this metric is as follows: 

$$
d = 1 - {|X \cap Y| \over |X \cup Y|}
$$

What this equation tells us is that given two strings, we want to find the number of characters they have in common and divide it by the number of characters in total. Then we subtract it by 1, so its is consistent with all other distance metric where the more dissimilar two strings are, the larger their distance metric. Consider the following toy example: 

In [0]:
documentX = "Republic of Korea"
documentY = "The Republic of Albania"

setX = set(documentX)
setY = set(documentY)

intersection = setX.intersection(setY)
union = setX.union(setY)

jaccard_dist = 1 - (len(intersection)/len(union))
jaccard_dist

0.33333333333333337

Of course, this can be applied to words of a string as well instead of individual characters. Consider the following code:  

In [0]:
import pandas as pd
import requests
import re

fed_1 = requests.get("http://dlsun.github.io/pods/data/federalist/1.txt").text.lower()
fed_2 = requests.get("http://dlsun.github.io/pods/data/federalist/2.txt").text.lower()

fed_1 = set([x for x in re.split("\n| |,|\.|\(|\)", fed_1) if x != '' and x != '"'])
fed_2 = set([x for x in re.split("\n| |,|\.|\(|\)", fed_2) if x != '' and x != '"'])

intersect = fed_1.intersection(fed_2)
union = fed_1.union(fed_2)

jaccard_dist = 1 - (len(intersect)/len(union))
jaccard_dist

0.7766895200783546

Fortunately for us, NLTK has a Jaccard Distance implementation for us to use so that we don't have to write all this code. 

In [0]:
jaccard_distance(fed_1, fed_2)

0.7766895200783546

On the surface, Jaccard distance seems a bit generic, but that is what makes this such a popular metric. Its only requirement is that the inputs are two sets which means that we can compare basically anything. One additional benefits of Jaccard distance is that duplicates does not affect the results since Jaccard distance operates on sets. 

## Building Similarity Matrix

Just like in our previous section, we want to build a similarity matrix that tells us which document is most similar to which other. To create a similarity matrix $S$, we want to ensure that $S_{ij}$ gives us the similarity between string i and string j. Note that in all the formulations we have given above, 0 represent identical strings, this can of course be inverted to fit your needs. 

The below code generates the a similarity matrix for the languages used for the translations of "We shall fight on the beaches" speech



In [0]:
import numpy as np

mtrx = np.zeros((df_speech.shape[0], df_speech.shape[0]))
for i in range(df_speech.shape[0]):
    for j in range(i + 1, df_speech.shape[0]):
        d = edit_distance(df_speech.iloc[i]["text"], df_speech.iloc[j]["text"])
        mtrx[i][j] = d
        mtrx[j][i] = d

mtrx = pd.DataFrame(mtrx, index=df_speech.index, columns=df_speech.index)
mtrx

language,english,spanish,german,portuguese,french,italian
language,,,,,,
english,0.0,276.0,334.0,281.0,328.0,293.0
spanish,276.0,0.0,338.0,130.0,280.0,220.0
german,334.0,338.0,0.0,344.0,368.0,345.0
portuguese,281.0,130.0,344.0,0.0,280.0,190.0
french,328.0,280.0,368.0,280.0,0.0,288.0
italian,293.0,220.0,345.0,190.0,288.0,0.0


This similarity matrix tells us how similar languages are to each other. It makes sense for this matrix to suggest that Spanish is more similar to Portuguese and Italian than English, French, or German. 

# Exercises 

1. Autocorrects determine if a word is correct by comparing it to an existing list of words. Download a dictionary of words from (https://raw.githubusercontent.com/dwyl/english-words/master/words_dictionary.json) and write a function that takes in a string and determine if there are any misspelling using the Hamming Distance.

2. Damerau-Levenshtein distance is often used to measure how similar two RNA/DNA sequences are. Download the data from (link) and determine which animal HIV came from.

    Note: It may take a while to calculate Damerau-Levenshtein distance for long sequences.



3. Use the Jaccard distance to create a similarity matrix for translations of Churchill's speeches. Can you see any advantages or disadvantages of using the Jaccard distance in this context?